# Os dados

Para coletar os dados de outros países, foram utilizadas as APIs dos sites:

- [About Corona](https://about-corona.net/dashboard/)
- [Europe RestFul](https://restcountries.eu/)

In [1]:

import requests
import pandas as pd

covid_api = 'https://corona-api.com/countries/'
rest_countries = 'https://restcountries.eu/rest/v2/alpha/'
country = 'IT' # Alpha-2 ISO3166

data_json =  requests.get(covid_api + country).json()
country = requests.get(covid_api + country).json()

N = country['data']['population']

print(country['data']['name'])


Italy


### Organizando os dados

In [2]:

from datetime import datetime

df = pd.DataFrame(data_json['data']['timeline'])
df = df.sort_values('date').reset_index()

from datetime import datetime, timedelta
df['date'] = [datetime.fromisoformat(f) for f in df['date']]
df = df.drop_duplicates(subset='date', keep = 'last')

# Criando o vetor de tempo
first_date = df['date'].iloc[0]
size_days = (df['date'].iloc[-1] - df['date'].iloc[0]).days
date_vec = [first_date + timedelta(days=k) for k in range(size_days)]

new_df = pd.DataFrame(date_vec, columns=['date'])
new_df = pd.merge(new_df, df, how='left', on= 'date')
new_df = new_df.drop(columns= ['index',  'updated_at', 'is_in_progress'])

for col in new_df.columns[1:]:
    new_df[col] = new_df[col].interpolate(method='polynomial', order=1)
df = new_df.dropna()

df.head()


,date,deaths,confirmed,active,recovered,new_confirmed,new_recovered,new_deaths
0,2020-01-30,0.0,2.000000,2.000000,0.0,2.000000,0.0,0.0
1,2020-01-31,0.0,2.000000,2.000000,0.0,0.000000,0.0,0.0
2,2020-02-01,0.0,2.142857,2.142857,0.0,0.142857,0.0,0.0
3,2020-02-02,0.0,2.285714,2.285714,0.0,0.285714,0.0,0.0
4,2020-02-03,0.0,2.428571,2.428571,0.0,0.428571,0.0,0.0


### Visualizando os dados

In [3]:

from bokeh.models   import Legend, ColumnDataSource, RangeTool, LinearAxis, Range1d, HoverTool
from bokeh.palettes import brewer, Inferno256
from bokeh.plotting import figure, show
from bokeh.layouts  import column
from bokeh.io       import output_notebook

output_notebook()

import numpy as np

# Criando os valores para legenda no plot
year =  [str(int(d.year)) for d in df['date'] ]
month = [("0"+str(int(d.month)))[-2:] for d in df['date'] ]
day =   [("0"+str(int(d.day)))[-2:] for d in df['date'] ]

# Criando a fonte de dados
source = ColumnDataSource(data={
    'Data'       : df['date'].values,
    'd': day, 'm': month, 'y': year,
    'Infectados Acc' : df['confirmed'].values,
    'Mortes'     : df['deaths'].values,
    'Ativo'     : df['active'].values,
    'Recuperados': df['recovered'].values
})


# Criando a figura
p = figure(plot_height=500,
           plot_width=600,
           x_axis_type="datetime", 
           tools="", 
           #y_axis_type="log",
           toolbar_location=None,
           title="Evolução do COVID - " + country['data']['name'])

# Preparando o estilo
p.grid.grid_line_alpha = 0
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
p.yaxis.axis_label = "Indivíduos"
p.xaxis.axis_label = "Dias"

# Incluindo as curvas
i_p = p.line(x='Data', y='Ativo',
             legend_label="Infectados", 
             line_cap="round", line_width=5, color="#c62828", source=source)
m_p = p.line(x='Data', y='Mortes',
             legend_label="Mortes",
             line_cap="round", line_width=5, color="#512da8", source=source)
c_p = p.line(x='Data', y='Infectados Acc',
             legend_label="Infectados Acc",
             line_cap="round", line_width=5, color="#0288d1", source=source)
r_p = p.line(x='Data', y='Recuperados',
             legend_label="Recuperados",
             line_cap="round", line_width=5, color="#388e3c", source=source)

# Colocando as legendas
p.legend.click_policy="hide"
# p.legend.location = "top_left"
p.legend.location = "top_left"

# Incluindo a ferramenta de hover
p.add_tools(HoverTool(
    tooltips=[
        ( 'Indivíduos', '$y{i}'),
        ( 'Data',       '@d/@m/@y' ),
    ],
    renderers=[
        m_p, i_p, c_p, r_p
    ]
))

show(p)


Loading BokehJS ...

## Verificando os dados

In [4]:

dif_I = np.diff(df['active'])
cum = []
cum.append(dif_I[0])

for k, i in enumerate(dif_I):
    cum.append(cum[-1] + i)
    
cum = np.array(cum)
cum += df['deaths'].to_numpy() + df['recovered'].to_numpy()

print("Erro entre casos acumulados e valores de confirmados: {}".format(
        round(sum((cum - df['confirmed'].values)**2 / len(cum)),2) ) ) 


Erro entre casos acumulados e valores de confirmados: 4.0


## Criando os dados SIR

In [5]:

I = df['active'].to_numpy()
R = df['recovered'].to_numpy()
M = df['deaths'].to_numpy()
S = N - R - I

# Creating the time vector
t = np.linspace(0, len(I), len(I))


Sd, Id, Md, Rd, td = S, I, M, R, t


# Estimando utilizando todos os dados

In [6]:

from models import *

dataset = dict(S=Sd, I=Id, R=Rd)

# Create the model
sir_model = ss.SIR(pop=N, focus=["S", "I", "R"])

# Adjust the parameters
sir_model.fit(dataset, td,
              search_pop=True,
              pop_sens=[0.001,0.01],
              beta_sens=[100000,100], 
              r_sens=[100000,100])


Loading BokehJS ...

	 ├─ S(0) ─ I(0) ─ R(0) ─  [60340326.0, 2.0, 0.0]
	 ├─ beta ─   1   r ─   0.14285714285714285
	 ├─ beta bound ─   1e-05  ─  100
	 ├─ r bound ─   1.4285714285714284e-06  ─  14.285714285714285
	 ├─ equation weights ─   [0.0017941069791369705, 1, 0.6955915519202226]
	 ├─ Running on ─  differential_evolution SciPy Search Algorithm
	 └─ Defined at:  67.3090510078555  ─  0.022488492441346073 



In [7]:

# Predict the model
sim_res = sir_model.predict((Sd[0],Id[0], Rd[0]), td)


In [8]:

p = figure(plot_height=500,
           plot_width=600, 
           tools="", 
           toolbar_location=None,
           title="Evolução do COVID - " + country['data']['name'])

# Preparando o estilo
p.grid.grid_line_alpha = 0
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
p.yaxis.axis_label = "Indivíduos"
p.xaxis.axis_label = "Dias"

p.line(t, I,  
       legend_label="Infectados", color="#c2185b", line_width=4)
p.line(t, R,  
       legend_label="Removidos", color="#0288d1", line_width=4)

# Show the results
p.line(td, sim_res[1],  
       legend_label="Infectados - Modelo", line_dash="dashed", color="#5e35b1", line_width=3)
p.line(td, sim_res[2],  
    legend_label="Removidos - Modelo", line_dash="dashed", color="#f57f17", line_width=3)
p.line(td, sim_res[0],  
      legend_label="Susceptible - Modelo", line_dash="dashed", color="#43a047", line_width=3)

show(p)


# Monte Carlo

In [ ]:

saved_param = {'r':[], 'beta':[], 'pop':[]}
saved_prediction = []

pred_t = np.array(range(int(td[-1]) + 120))

for i in range(35,len(I),1):

    dataset = dict(S=Sd[:i], I=Id[:i], R=Rd[:i])
    td_ = td[:i]

    # Create the model
    sir_model = ss.SIR(pop=N, focus=["S", "I", "R"], verbose = False)

    # Adjust the parameters
    sir_model.fit(dataset, td_,
                  search_pop=True,
                  pop_sens=[0.001,0.01],
                  beta_sens=[100000,100], 
                  r_sens=[100000,100])
    
    saved_param['r'].append(sir_model.parameters[0])
    saved_param['beta'].append(sir_model.parameters[1])
    saved_param['pop'].append(sir_model.parameters[2])
    
    saved_prediction.append(sir_model.predict((Sd[0],Id[0], Rd[0]), pred_t))
    

In [ ]:

p2 = figure(plot_height=500,
           plot_width=600, 
           tools="", 
           toolbar_location=None,
           title="Evolução do COVID - " + country['data']['name'])

x = t[35:]
index = 20

# Preparando o estilo
p2.grid.grid_line_alpha = 0
p2.ygrid.band_fill_color = "olive"
p2.ygrid.band_fill_alpha = 0.1
p2.yaxis.axis_label = "Indivíduos"
p2.xaxis.axis_label = "Dias"

# Incluindo as curvas
for data in saved_prediction[:45]:
    p2.line(pred_t, data[1],
           legend_label="Previsão Infectados", 
           line_cap="round", line_width=4, color="#42a5f5", line_alpha = 0.1)
    
p2.line(td, Id,
       legend_label="Infectados", 
       line_cap="round", line_width=5, color="#c62828")

# Colocando as legendas
p2.legend.click_policy="hide"
p2.legend.location = "top_right"

show(p2)


In [ ]:

p3 = figure(plot_height=500,
           plot_width=600, 
           tools="", 
           toolbar_location=None,
           title="Evolução do COVID - " + country['data']['name'])

x = t[35:]
index = 20
plot_all = True

# Preparando o estilo
p3.grid.grid_line_alpha = 0
p3.ygrid.band_fill_color = "olive"
p3.ygrid.band_fill_alpha = 0.1
p3.yaxis.axis_label = "Indivíduos"
p3.xaxis.axis_label = "Dias"

# Incluindo as curvas
for data in saved_prediction[10:45]:
    p3.line(pred_t, data[1],
           legend_label="Previsão Infectados", 
           line_cap="round", line_width=4, color="#42a5f5", line_alpha = 0.1)

    p3.line(pred_t, data[0],
       legend_label="Previsão Suscetiveis", 
       line_cap="round", line_width=4, color="#ff5722", line_alpha = 0.07)
    
    p3.line(pred_t, data[2],
       legend_label="Previsão Recuperados", 
       line_cap="round", line_width=4, color="#9c27b0", line_alpha = 0.07)
    
    
p3.line(td, Id,
       legend_label="Infectados", 
       line_cap="round", line_width=5, color="#005cb2", line_dash = 'dashed')

if plot_all:
    p3.line(td, Rd,
           legend_label="Recuperados", 
           line_cap="round", line_width=5, color="#5e35b1", line_dash = 'dashed')

    p3.line(td, N*saved_param['pop'][-10] - Rd - Id,
           legend_label="Suscetiveis", 
           line_cap="round", line_width=5, color="#b71c1c", line_dash = 'dashed')

# Colocando as legendas
p3.legend.click_policy="hide"
p3.legend.location = "top_right"

show(p3)
